In [1]:
import flwr as fl
import tensorflow as tf
from tensorflow import keras
import sys
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

In [11]:
def preprocess_data(train_samples_per_class=100, test_samples_per_class=20):
    # Load the MNIST dataset
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

    # Initialize empty lists to store the selected samples for training and testing
    x_train_subset = []
    y_train_subset = []
    x_test_subset = []
    y_test_subset = []

    # Create a list of unique class labels
    unique_classes = np.unique(y_train)

    # Iterate through each class and select samples for training and testing
    for class_label in unique_classes:
        # Get the indices of samples for the current class
        class_indices = np.where(y_train == class_label)[0]

        # Randomly shuffle the indices to ensure randomness
        np.random.shuffle(class_indices)

        # Select the desired number of samples for training and testing from the current class
        selected_indices_train = class_indices[:train_samples_per_class]
        selected_indices_test = class_indices[train_samples_per_class:train_samples_per_class+test_samples_per_class]

        # Add the selected training samples to the training subset
        x_train_subset.extend(x_train[selected_indices_train])
        y_train_subset.extend(y_train[selected_indices_train])

        # Add the selected testing samples to the testing subset
        x_test_subset.extend(x_train[selected_indices_test])
        y_test_subset.extend(y_train[selected_indices_test])

    # Convert the lists to NumPy arrays
    x_train_subset = np.array(x_train_subset)
    y_train_subset = np.array(y_train_subset)
    x_test_subset = np.array(x_test_subset)
    y_test_subset = np.array(y_test_subset)

    # Preprocess the data by normalizing it
    x_train_subset, x_test_subset = x_train_subset[..., np.newaxis] / 255.0, x_test_subset[..., np.newaxis] / 255.0

    return x_train_subset, y_train_subset, x_test_subset, y_test_subset

x_train_subset, y_train_subset, x_test_subset, y_test_subset = preprocess_data(train_samples_per_class=100, test_samples_per_class=20)


In [3]:
x_train_subset.shape,x_test_subset.shape

((1000, 28, 28, 1), (200, 28, 28, 1))

In [4]:
# Define the client model
def create_client_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(256, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])
    return model

In [5]:
from tensorflow.keras.utils import plot_model
import visualkeras
model=create_client_model()
plot_model(model, to_file='model_architecture.png', show_shapes=True)
visualkeras.layered_view(model)

You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) for plot_model to work.


In [6]:
model.get_weights()

[array([[-0.05644973,  0.03650607, -0.08105209, ..., -0.02430679,
         -0.00801607,  0.02015902],
        [-0.02332864,  0.01300106, -0.06357176, ...,  0.06665432,
         -0.0385451 , -0.07755464],
        [-0.01035547, -0.05374467,  0.05990107, ...,  0.02780173,
         -0.05767986,  0.00500762],
        ...,
        [-0.03176086, -0.02040316,  0.02116834, ..., -0.00333721,
         -0.05005827, -0.07945918],
        [ 0.0052898 ,  0.05675804, -0.03929036, ...,  0.05457688,
         -0.05346359, -0.07951298],
        [-0.06578673, -0.02806577,  0.06810481, ...,  0.00692996,
         -0.08051791, -0.01169664]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [7]:
PORT=5010

In [ ]:
# Define the client logic
class MnistClient(fl.client.NumPyClient):
    def __init__(self, learning_rate=0.001, batch_size=32):
        self.model = create_client_model()
        self.x_train, self.y_train, self.x_test, self.y_test = preprocess_data()
        self.learning_rate = learning_rate
        self.batch_size = batch_size

    def get_parameters(self,config=None):
        return self.model.get_weights()
    
    
    def fit(self, parameters, config=None):
        optimizer = tf.keras.optimizers.Adam(learning_rate=self.learning_rate)
        batch_size = self.batch_size
        self.model.set_weights(parameters)
        self.model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"])
        self.model.fit(self.x_train, self.y_train, batch_size=batch_size, epochs=1, verbose=1)
        return self.model.get_weights(), len(self.x_train), {}


    def evaluate(self, parameters, config=None):
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.x_test, self.y_test, verbose=0)
        return loss, len(self.x_test), {"accuracy": accuracy}

# Start the Flower client
fl.client.start_numpy_client(server_address='localhost:'+str(PORT), client=MnistClient(),grpc_max_message_length = 1024*1024*1024)


INFO flwr 2023-11-03 17:40:00,794 | grpc.py:49 | Opened insecure gRPC connection (no certificates were passed)
DEBUG flwr 2023-11-03 17:40:00,817 | connection.py:42 | ChannelConnectivity.IDLE
DEBUG flwr 2023-11-03 17:40:00,823 | connection.py:42 | ChannelConnectivity.READY


32/32 [==============================] - 2s 6ms/step - loss: 1.5122 - accuracy: 0.5490


In [9]:
cl=MnistClient()
cl.get_parameters()

[array([[-0.03749718, -0.07887228, -0.02979475, ...,  0.00540863,
         -0.05379724,  0.01651713],
        [-0.02821155,  0.00611251,  0.02285513, ...,  0.06443251,
          0.03561399,  0.0456214 ],
        [-0.04603993,  0.01479301,  0.00112346, ..., -0.07992301,
         -0.06754994, -0.03844707],
        ...,
        [ 0.0259298 ,  0.03359801, -0.06574246, ..., -0.01261174,
         -0.01736881,  0.00695976],
        [-0.0426014 ,  0.06605002,  0.05210688, ...,  0.02176531,
         -0.01500161,  0.06764663],
        [-0.00666784, -0.06883423,  0.0539613 , ...,  0.03659391,
          0.0556917 , -0.06274236]], dtype=float32),
 array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.